In [1]:
import pandas as pd
import numpy as np
from scapy.all import *

D:\Program\Anaconda\envs\dip2022\lib\site-packages\scapy\layers\ipsec.py:471: CryptographyDeprecationWarning: Blowfish has been deprecated
  cipher=algorithms.Blowfish,
D:\Program\Anaconda\envs\dip2022\lib\site-packages\scapy\layers\ipsec.py:485: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher=algorithms.CAST5,


# Chunk Detection

In [2]:

from math import *
import scapy.layers as layers
IP = layers.inet.IP

GET_THRESH = 300 # bytes
DOWN_THRESH = 300  # bytes
VIDEO_CHUNK_GETSIZE = 700 # bytes
AUDIO_CHUNK_GETSIZE = 600 # bytes
NETINFO_NUM = 25

class Chunk():
    def __init__(self, start_time = 0, server_ip='',ttfb = 0, download_time = 0, slack_time = 0, get_size=0, chunk_size = 0, type="", protocol=""):
        self.start_time = start_time
        self.server_ip = server_ip
        self.ttfb = ttfb
        self.download_time = download_time
        self.slack_time = slack_time
        self.get_size = get_size
        self.chunk_size = chunk_size
        self.type = type
        self.protocol = protocol
    def __str__(self):
        return f"{self.start_time} {self.ttfb} {self.download_time} {self.slack_time} {self.get_size} {self.chunk_size} {self.type} {self.protocol}"
    def __repr__(self):
        return f"{self.start_time} {self.ttfb} {self.download_time} {self.slack_time} {self.get_size} {self.chunk_size} {self.type} {self.protocol}"


def isUplink(p):
    # IP = scapy.layers.inet.IP
    return p[IP].src.startswith('192.168.')

def detectAV(c):
    if abs(c.get_size-VIDEO_CHUNK_GETSIZE) > abs(c.get_size-AUDIO_CHUNK_GETSIZE):
        flag=0
    else:
        flag=1
    if c.chunk_size<=80*1024:
        flag=2
    return flag

def chunkDetection(filename):
    a = rdpcap(filename)
    meta_time = float(a[0].time)
    chunk = {}
    chunks = []
    downFlag = {}
    # IP = scapy.layers.inet.IP
    for p in a:
        if p.haslayer(IP):
            ipSrc=p[IP].src
            ipDst=p[IP].dst
            pLen=p[IP].len
            pHdr=p[IP].ihl*4
            ip_time=float(p.time)
            if isUplink(p) and pLen-pHdr > GET_THRESH:
                if ipDst in chunk:
                    chunk[ipDst].slack_time = ip_time - chunk[ipDst].download_time
                    avFlag=detectAV(chunk[ipDst])
                    if avFlag==0:
                        # chunk[ipDst].type='a'
                        chunks.append(chunk[ipDst])
                    elif avFlag==1:
                        # chunk[ipDst].type='v'
                        chunks.append(chunk[ipDst])
                    else:
                        chunk.pop(ipDst)
                        downFlag.pop(ipDst)
                chunk[ipDst] = Chunk(start_time=ip_time, get_size=pLen-pHdr, server_ip=ipDst)
                downFlag[ipDst] = False
            elif not isUplink(p) and pLen > DOWN_THRESH:
                if ipSrc in chunk:
                    if not downFlag[ipSrc]:
                        chunk[ipSrc].ttfb = ip_time
                        downFlag[ipSrc] = True
                    chunk[ipSrc].download_time = ip_time
                    chunk[ipSrc].chunk_size += pLen - pHdr
                    chunk[ipSrc].protocol = p.proto
    
    for c in chunk.values():
        avFlag=detectAV(c)
        if avFlag==0:
            # c.type='a'
            chunks.append(c)
        elif avFlag==1:
            # c.type='v'
            chunks.append(c)
    return meta_time, chunks

filename = 'RequetDataSetNew/A0/PCAP_FILES/baseline_Jan17_exp_31.pcap'
meta_time, chunks=chunkDetection(filename)

In [3]:
print(chunks)

[1516209610.386307 1516209610.410726 1516209612.950441 0.02435302734375 1247 624049  6, 1516209611.082223 1516209611.087837 1516209614.529326 0.03212308883666992 310 2129190  6, 1516209614.635162 1516209614.638517 1516209615.156041 0.029316186904907227 316 1587091  6, 1516209615.259879 1516209615.281233 1516209616.121418 0.03412294387817383 306 2209745  6, 1516209616.214358 1516209616.225509 1516209616.951287 0.07074189186096191 310 2209681  6, 1516209617.060617 1516209617.064965 1516209617.69758 0.04791998863220215 313 1627930  6, 1516209612.974794 1516209613.085303 1516209618.181602 0.5422348976135254 1365 132789  6, 1516209617.796378 1516209617.799761 1516209619.590904 0.029239892959594727 309 1536081  6, 1516209618.723837 1516209618.758569 1516209621.077217 0.0068759918212890625 1246 450948  6, 1516209621.084093 1516209621.103908 1516209621.48225 0.06220293045043945 1255 123272  6, 1516209619.657107 1516209619.661844 1516209621.994784 0.03506898880004883 316 1573298  6, 1516209622.

In [4]:
import time
start = time.time()
filename = 'RequetDataSetNew/A0/PCAP_FILES/baseline_Jan17_exp_31.pcap'
pacpfile = rdpcap(filename)
end = time.time()
print(f"Use Time:{end - start}")
print(f'lenth:{len(pacpfile)}')

Use Time:25.146851778030396
lenth:84540


In [19]:
GET_THRESH = 300  # bytes
DOWN_THRESH = 300  # bytes
VIDEO_CHUNK_GETSIZE = 700  # bytes
AUDIO_CHUNK_GETSIZE = 600  # bytes
import scapy.layers as layers

def isUplink(p):
    return p[IP].src.startswith('192.168.')
def detectAV(c):
    if abs(c.get_size-VIDEO_CHUNK_GETSIZE) > abs(c.get_size-AUDIO_CHUNK_GETSIZE):
        flag=0
    else:
        flag=1
    if c.chunk_size<=80*1024:
        flag=2
    return flag
chunk = {}
chunks = []
downFlag = {}
#  用show显示包的内容
count=0
# print(pacpfile[0].show())
IP = layers.inet.IP
for p in pacpfile[0:13]:
    if p.haslayer(IP):
        ipSrc = p[IP].src
        ipDst = p[IP].dst
        pLen = p[IP].len
        pHdr = p[IP].ihl*4
        ip_time = float(p.time)
        end_time = ip_time
        # print(p.show())
        if isUplink(p) and pLen-pHdr > GET_THRESH:
            print(f'ipSrc:{ipSrc}, ipDst:{ipDst}, pLen:{pLen}, pHdr:{pHdr}, ip_time:{ip_time}, end_time:{end_time}')
            if ipDst in chunk:
                chunk[ipDst].slack_time = ip_time - chunk[ipDst].download_time
                avFlag = detectAV(chunk[ipDst])
                if avFlag == 0:
                    # chunk[ipDst].type='a'
                    chunks.append(chunk[ipDst])
                elif avFlag == 1:
                    # chunk[ipDst].type='v'
                    chunks.append(chunk[ipDst])
                else:
                    chunk.pop(ipDst)
                    downFlag.pop(ipDst)
            chunk[ipDst] = Chunk(start_time=ip_time, get_size=pLen-pHdr, server_ip=ipDst)
            downFlag[ipDst] = False
        elif not isUplink(p) and pLen > DOWN_THRESH:
            if ipSrc in chunk:
                if not downFlag[ipSrc]:
                    chunk[ipSrc].ttfb = ip_time
                    downFlag[ipSrc] = True
                chunk[ipSrc].download_time = ip_time
                chunk[ipSrc].chunk_size += pLen - pHdr
                chunk[ipSrc].protocol = p.proto
for c in chunk.values():
    avFlag=detectAV(c)
    if avFlag==0:
        # c.type='a'
        chunks.append(c)
    elif avFlag==1:
        # c.type='v'
        chunks.append(c)
# f"{self.start_time} {self.ttfb} {self.download_time} {self.slack_time} {self.get_size} {self.chunk_size} {self.type} {self.protocol}"
print(chunk)

ipSrc:192.168.1.190, ipDst:172.217.12.142, pLen:379, pHdr:20, ip_time:1516209607.012159, end_time:1516209607.012159
ipSrc:192.168.1.190, ipDst:172.217.12.142, pLen:510, pHdr:20, ip_time:1516209607.05867, end_time:1516209607.05867
{'172.217.12.142': 1516209607.05867 0 0 0 490 0  }


In [16]:
print(chunk)

{'172.217.10.102': 1516209607.812766 0 0 0 448 0  , '23.206.171.9': 1516209624.306796 1516209624.312524 1516209625.674749 0 319 892970  6, '172.217.10.226': 1516209626.317585 1516209626.340094 1516209626.340094 0 536 879  6, '172.217.15.100': 1516209626.419784 0 0 0 348 0  , '172.217.6.194': 1516209636.556193 0 0 0 485 0  , '172.217.10.67': 1516209671.121419 0 0 0 1400 0  , '172.217.11.10': 1516209717.980839 1516209718.011638 1516209718.012025 0 858 2694  6, '172.217.11.3': 1516209966.46761 1516209966.510818 1516209966.511161 0 392 1391  6, '173.194.7.39': 1516210073.98207 1516210073.997257 1516210075.529079 0 1373 1981955  6, '172.217.6.206': 1516210169.404179 1516210169.408949 1516210169.456366 0 397 49286  6, '172.217.12.142': 1516210196.055187 0 0 0 590 0  , '172.217.11.19': 1516210198.297022 0 0 0 367 0  }


In [15]:
columns = ['start_time', 'server_ip', 'type', 'ttfb', 'download_time', 'get_size', 'chunk_size', 'rel_start_time', 'rel_end_time']
df_chunk=pd.DataFrame(columns=columns)


for c in chunks:
    start_time = c.start_time
    server_ip = c.server_ip
    type = c.type
    ttfb = c.ttfb - c.start_time
    download_time = c.download_time - c.start_time
    get_size = c.get_size
    chunk_size = c.chunk_size
    rel_start_time = start_time-meta_time
    rel_end_time = c.download_time - meta_time
    s = pd.Series([start_time, server_ip, type, ttfb, download_time, get_size, chunk_size, rel_start_time, rel_end_time], index=columns)
    df_chunk.loc[len(df_chunk)] = s

av_getsize_gap = df_chunk.get_size.mean()

for i in range(len(df_chunk)):
    df_chunk.loc[i, 'type'] = 'a' if df_chunk.loc[i, 'get_size'] < av_getsize_gap else 'v'

df_chunk.sort_values(by='start_time', inplace=True)
df_chunk.to_csv('results/chunk_detection/A0/baseline_Jan17_exp_30.csv', index=False)

In [22]:
df_chunk

NameError: name 'df_chunk' is not defined

In [18]:
start_time = df_chunk['start_time'].min()
end_time = df_chunk['start_time'].max()

columns=['timestamp']+[str(i) for i in range(1,121)]

df_120features=pd.DataFrame(columns=columns)

for t_msec in range(int(start_time*1000), int(end_time*1000+100), 100):
    s = [t_msec]
    t = t_msec/1000
    for w in range(1, 21):
        period = w*10.0
        if t-period<start_time:
            total_number_of_chunks_v = -1
            avg_chunk_size_v = -1
            download_time_v = -1
            total_number_of_chunks_a = -1
            avg_chunk_size_a = -1
            download_time_a = -1
        else:
            total_number_of_chunks_v = df_chunk[(df_chunk['type']=='v') & (df_chunk['start_time']>t-period) & (df_chunk['start_time']<t)].shape[0]
            avg_chunk_size_v = df_chunk[(df_chunk['type']=='v') & (df_chunk['start_time']>t-period) & (df_chunk['start_time']<t)]['chunk_size'].mean()
            download_time_v = df_chunk[(df_chunk['type']=='v') & (df_chunk['start_time']>t-period) & (df_chunk['start_time']<t)]['download_time'].sum()
            total_number_of_chunks_a = df_chunk[(df_chunk['type']=='a') & (df_chunk['start_time']>t-period) & (df_chunk['start_time']<t)].shape[0]
            avg_chunk_size_a = df_chunk[(df_chunk['type']=='a') & (df_chunk['start_time']>t-period) & (df_chunk['start_time']<t)]['chunk_size'].mean()
            download_time_a = df_chunk[(df_chunk['type']=='a') & (df_chunk['start_time']>t-period) & (df_chunk['start_time']<t)]['download_time'].sum()
        s += [total_number_of_chunks_v, avg_chunk_size_v, download_time_v, total_number_of_chunks_a, avg_chunk_size_a, download_time_a]
    
    df_120features.loc[len(df_120features)] = s

df_120features.to_csv('results/120features/A0/baseline_Jan17_exp_30.csv', index=False)

In [12]:
start_time = df_chunk['start_time'].min()
end_time = df_chunk['start_time'].max()

columns=['timestamp']+[str(i) for i in range(1,121)]

df_120features=pd.DataFrame(columns=columns)

df_audio_chunk=df_chunk[df_chunk.type=='a']
df_video_chunk=df_chunk[df_chunk.type=='v']

audio_chunk_prev = {'chunk_size':[0], 'download_time':[0]}
video_chunk_prev = {'chunk_size':[0], 'download_time':[0]}

for i in range(len(df_audio_chunk)):
    audio_chunk_prev['chunk_size'].append(audio_chunk_prev['chunk_size'][i] + df_audio_chunk.loc[i, 'chunk_size'])
    audio_chunk_prev['download_time'].append(audio_chunk_prev['download_time'][i] + df_audio_chunk.loc[i, 'download_time'])

for i in range(len(df_video_chunk)):
    video_chunk_prev['chunk_size'].append(video_chunk_prev['chunk_size'][i] + df_video_chunk.loc[i, 'chunk_size'])
    video_chunk_prev['download_time'].append(video_chunk_prev['download_time'][i] + df_video_chunk.loc[i, 'download_time'])


for t_msec in range(int(start_time*1000), int(end_time*1000+100), 100):
    s = [t_msec]
    t = t_msec/1000
    for w in range(1, 21):
        l_audio = 0
        r_audio = 0
        l_video = 0
        r_video = 0
        period = w * 10.0
        l_t = t - period
        if l_t<start_time:
            total_number_of_chunks_v = -1
            avg_chunk_size_v = -1
            download_time_v = -1
            total_number_of_chunks_a = -1
            avg_chunk_size_a = -1
            download_time_a = -1
        else:
            while df_audio_chunk.loc[l_audio, 'start_time'] < l_t:
                l_audio += 1
            while df_audio_chunk.loc[r_audio, 'start_time'] <= t:
                r_audio += 1
            while df_video_chunk.loc[l_video, 'start_time'] < l_t:
                l_video += 1
            while df_video_chunk.loc[r_video, 'start_time'] <= t:
                r_video += 1
            total_number_of_chunks_v = r_video - l_video 
            avg_chunk_size_v = video_chunk_prev['chunk_size'][r_video] - video_chunk_prev['chunk_size'][l_video]
            download_time_v = video_chunk_prev['download_time'][r_video] - video_chunk_prev['download_time'][l_video]
            total_number_of_chunks_a = r_audio - l_audio
            avg_chunk_size_a = audio_chunk_prev['chunk_size'][r_audio] - audio_chunk_prev['chunk_size'][l_audio]
            download_time_a = audio_chunk_prev['download_time'][r_audio-1] - audio_chunk_prev['download_time'][l_audio-1]
        s += [total_number_of_chunks_v, avg_chunk_size_v, download_time_v, total_number_of_chunks_a, avg_chunk_size_a, download_time_a]
    
    df_120features.loc[len(df_120features)] = s

df_120features.to_csv('results/120features/A0/baseline_Jan17_exp_30.csv', index=False)

KeyError: 0